In [6]:
# strategy3

import ccxt 
import telegram
import time
from datetime import datetime
import pandas as pd
import numpy as np
import talib
import math
from binance.client import Client
from binance_f import RequestClient

api_key = '-'
api_secret = '-'

client = Client(api_key=api_key, api_secret=api_secret)
request_client = RequestClient(api_key = api_key, secret_key = api_secret, url='https://fapi.binance.com')

with open("api_test1.txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret  = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, 
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})

bot = telegram.Bot(token='-')
chat_id = '-'

while True:

    sol = binance.fetch_ohlcv(
        symbol = "SOL/BUSD",
        timeframe='1m',
        since=None,
        limit=1000
    )

    df = pd.DataFrame(sol, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)

    open_orders = binance.fetch_open_orders(
    symbol="SOL/BUSD"
    )

    # ma60
    # df['ma60'] = talib.MA(np.asarray(df['close']), 60)
    # df['ma60_incli'] = df.ma60 - df.ma60.shift()
    # df['ma60_degrees'] = df['ma60_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

    # ma100
    df['ma100'] = talib.MA(np.asarray(df['close']), 100)
    df['ma100_incli'] = df.ma100 - df.ma100.shift()
    df['ma100_degrees'] = df['ma100_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

    # ma120
    df['ma120'] = talib.MA(np.asarray(df['close']), 120)
    df['ma120_incli'] = df.ma120 - df.ma120.shift()
    df['ma120_degrees'] = df['ma120_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

    # ma200
    df['ma200'] = talib.MA(np.asarray(df['close']), 200)
    df['ma200_incli'] = df.ma200 - df.ma200.shift()
    df['ma200_degrees'] = df['ma200_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

    # ma240
    df['ma240'] = talib.MA(np.asarray(df['close']), 240)
    df['ma240_incli'] = df.ma240 - df.ma240.shift()
    df['ma240_degrees'] = df['ma240_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

    macd, macdsignal, macdhist = talib.MACD(np.asarray(df['close']), 12, 26, 9)

    df['macd'] = macd
    df['signal'] = macdsignal
    # df['macdhist'] = macdhist

    # ma60 = df['ma60']
    ma100 = df['ma100']
    ma120 = df['ma120']
    ma200 = df['ma200']
    ma240 = df['ma240']

    # ma60_deg_r = round(df['ma60_degrees'], 5)
    ma100_deg_r = round(df['ma100_degrees'], 5)
    ma120_deg_r = round(df['ma120_degrees'], 5)
    ma200_deg_r = round(df['ma200_degrees'], 5)
    ma240_deg_r = round(df['ma240_degrees'], 5)
    macd_r = round(df['macd'], 5)
    macdsignal_r = round(df['signal'], 5)

    # 롱 주문조건
    if (len(open_orders)==0) & (ma100_deg_r[-2]>ma120_deg_r[-2]>ma200_deg_r[-2]>ma240_deg_r[-2]>0) & (ma100[-2]>ma120[-2]>ma200[-2]>ma240[-2]) &(macd_r[-2]> 0) & (macdsignal_r[-2]> 0):
    
        orders = [None] * 3
        take_porfit_l = round((df['close'][-2] + df['close'][-2]*0.02), 2)
        stop_loss_l = round((df['close'][-2] - df['close'][-2]*0.005), 2)

        #market price
        orders[0] = binance.create_order(
            symbol = 'SOL/BUSD',
            type='MARKET',
            side='buy',
            amount=1
        )
        
        # take profit
        orders[1] = client.futures_create_order(
            symbol="SOLBUSD",
            type="LIMIT",
            timeInForce='GTC',
            price= take_porfit_l,
            side='SELL',
            quantity=1
)
        
        # stop loss
        orders[2] = binance.create_order(
            symbol="SOL/BUSD",
            type="STOP_MARKET",
            side="sell",
            amount=1,
            params={'stopPrice': stop_loss_l}
        )

        l_text = {
            'buy': 'buy',
            'macd_r': macd_r[-2],
            'macdsignal_r' : macdsignal_r[-2],
            'ma100' : ma100_deg_r[-2],
            'ma120' : ma120_deg_r[-2],
            'ma200' : ma200_deg_r[-2],
            'ma240' : ma240_deg_r[-2],
            'TP':take_porfit_l,
            'SL':stop_loss_l
        }
        bot.sendMessage(chat_id=chat_id, text=l_text)
        print('=====BUY=====', l_text)
        # macd_cro = 0

    # 숏 주문조건
    elif (len(open_orders)==0) & (ma100_deg_r[-2]<ma120_deg_r[-2]<ma200_deg_r[-2]<ma240_deg_r[-2]<0) & (ma100[-2]<ma120[-2]<ma200[-2]<ma240[-2]) &(macd_r[-2]< 0) & (macdsignal_r[-2]<0):
    
        orders = [None] * 3
        take_porfit_s = round((df['close'][-2] - df['close'][-2]*0.02), 2)
        stop_loss_s = round((df['close'][-2] + df['close'][-2]*0.005), 2)

        #market price
        orders[0] = binance.create_order(
            symbol = 'SOL/BUSD',
            type='MARKET',
            side='sell',
            amount=1
        )
        
        # take profit
        orders[1] = client.futures_create_order(
            symbol="SOLBUSD",
            type="LIMIT",
            timeInForce='GTC',
            price= take_porfit_s,
            side='BUY',
            quantity=1
)
        
        # stop loss
        orders[2] = binance.create_order(
            symbol="SOL/BUSD",
            type="STOP_MARKET",
            side="buy",
            amount=1,
            params={'stopPrice': stop_loss_s}
        )

        s_text = {
            'sell': 'sell',
            'macd_r': macd_r[-2],
            'macdsignal_r' : macdsignal_r[-2],
            'ma100' : ma100_deg_r[-2],
            'ma120' : ma120_deg_r[-2],
            'ma200' : ma200_deg_r[-2],
            'ma240' : ma240_deg_r[-2],
            'TP':take_porfit_s,
            'SL':stop_loss_s
        }
        bot.sendMessage(chat_id=chat_id, text=s_text)
        print('=====SELL=====', s_text)
        # macd_cro = 0

    if len(open_orders)==1:
        result = request_client.auto_cancel_all_orders(symbol='SOLBUSD', countdownTime=500)
        bot.sendMessage(chat_id=chat_id, text="Cancel all orders")

    time.sleep(60)

# TRADING ON! v0.6


=====SELL===== {'sell': 'sell', 'macd_r': -0.00388, 'macdsignal_r': -0.00054, 'ma100': -0.00054, 'ma120': -0.0004, 'ma200': -0.00025, 'ma240': -0.00011, 'TP': 30.36, 'SL': 31.13}
====== Request ======
header:{'client_SDK_Version': 'binance_futures-0.0.1-py3.9', 'Content-Type': 'application/json', 'X-MBX-APIKEY': 'jGsDOVAOk6c2U1wDqNgyQAVhyq0W7dX0ot1tLbhvMxk9XjtJmLxltIuBLBn9m9R5'}
host:https://fapi.binance.com
json_parser:None
method:POST
post_body:{}
url:/fapi/v1/countdownCancelAll?symbol=SOLBUSD&countdownTime=500&recvWindow=60000&timestamp=1662197660263&signature=8ebff891e62d82c775c91c6c96e7ec5074e7cbd2fba822e395643225f04933be
=====SELL===== {'sell': 'sell', 'macd_r': -0.02902, 'macdsignal_r': -0.02351, 'ma100': -0.00064, 'ma120': -0.0005, 'ma200': -0.0003, 'ma240': -0.0002, 'TP': 30.33, 'SL': 31.1}
====== Request ======
header:{'client_SDK_Version': 'binance_futures-0.0.1-py3.9', 'Content-Type': 'application/json', 'X-MBX-APIKEY': 'jGsDOVAOk6c2U1wDqNgyQAVhyq0W7dX0ot1tLbhvMxk9XjtJmLxlt